### このノート
- SMT（背景論理付きSATソルバー）を使いたい
  - Z3を使ってみる
  
- data : SNP_df55.csv, y5.csv

In [72]:
#`brew install z3`をterminalで実行した

In [59]:
import itertools
import pandas as pd
import numpy as np
from z3 import *

In [60]:
X55 = pd.read_csv("../input/SNP_df55.csv", sep=',', index_col=0)
y5 = pd.read_csv('../input/y5.csv', sep=',', index_col=0)['y']

In [61]:
num_samples = X55.shape[0]
num_cols = X55.shape[1]

In [62]:
selected_col_num2 = 2

In [63]:
y1_sum = sum(y5)
y1_sum

3

In [64]:
def featrure_InnerProduct_sel_mul(X, y, sel_col_num):
    inner_product_list = [np.dot(X[each_col], y) for each_col in X.columns] #内積のリスト
    sort_inner_product_list = sorted(inner_product_list ,reverse=True)
    select_cols = []
    for i in range(sel_col_num):
        iindex = inner_product_list.index(sort_inner_product_list[i])
        select_cols.append(X.columns[iindex])
    
    else_cols = list(X.columns)
    for col in select_cols:
        else_cols.remove(col)
    
    return select_cols, else_cols

In [65]:
select_cols, else_cols = featrure_InnerProduct_sel_mul(X55, y5, selected_col_num2)

In [66]:
ys = [BitVec("y%d" % i, 1) for i in range(num_samples)]

In [67]:
s = Solver()

In [68]:
for sel in select_cols:
    for els in else_cols:
        formu_plus = 0
        formu_minu = 0
        y_sum = 0
        for sum_num in range(num_samples):
            formu_plus += X55[sel][sum_num]*ys[sum_num]
            formu_minu += X55[els][sum_num]*ys[sum_num]
            y_sum += ys[sum_num]
        s.add(formu_plus - formu_minu >= 0)
        s.add(y_sum == y1_sum)

In [69]:
#別の方法で試そうとした形跡

#論理And
#And([P1, P2, ...])
#P_list = 
#s.add(And(P_list))

In [70]:
s.check()
m = s.model()

In [71]:
for v in ys:
    print(m[v].as_long())

0
1
1
1
0
